In [ ]:
import pandas as pd
import time
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


In [ ]:
# === Excel 配置 ===
EXCEL_PATH = "birthday.xlsx"  # 改成本地文件名
SHEET_NAME = "Sheet2"
BIRTHDAY_MONTH = 12  # 目标月份
USER = "your account"  # 你的账号
PASS = "your password"  # 你的密码


In [ ]:

# === 读取数据 ===
df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME)

print("列名为：", list(df.columns))

# === 修改为你的真实列名 ===
name_col = "姓名"
email_col = "邮箱"
month_col = "月"
day_col = "日"

# === 筛选 12 月 ===
df_dec = df[df[month_col] == BIRTHDAY_MONTH].copy().reset_index(drop=True)

df_dec


In [ ]:
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

TEMPLATE_URL = "https://peeryihengzhiyou.lingxi360.com/drag_templates/index?id=LXE481GMdIQi&did=112690"

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 5)

driver.get(TEMPLATE_URL)

print("🌐 已打开模板链接，检测是否需要登录中...")

def auto_login_if_needed():
    try:
        email_input = wait.until(
            EC.presence_of_element_located((By.ID, "login-email"))
        )
        print("🔐 检测到登录页面，开始自动登录")

        password_input = driver.find_element(By.ID, "login-password")
        login_btn = driver.find_element(By.ID, "pwd_login_btn")

        # 从环境变量中读取账号密码


        if not USER or not PASS:
            raise RuntimeError("未检测到环境变量 LINGXI_USER / LINGXI_PASS，请先设置。")

        email_input.clear()
        email_input.send_keys(USER)
        password_input.clear()
        password_input.send_keys(PASS)

        login_btn.click()
        print("✅ 已提交登录信息，等待登录完成...")
        
        # 等页面跳转至模板
        wait.until(EC.url_contains("drag_templates"))
        print("✅ 登录成功，已进入模板页面")

    except Exception:
        print("✅ 未检测到登录框，可能已自动登录")

# 调用自动登录检测
auto_login_if_needed()


In [ ]:
def wait_for_loading_to_disappear(timeout=60):
    """
    等待常见 loading / 遮罩层 消失
    """
    print("⏳ 等待页面加载动画消失...")

    possible_loaders = [
        ".loading",
        ".loader",
        ".spinner",
        ".mask",
        ".overlay",
        ".el-loading-mask",
        ".ant-spin",
        ".v-loading",
        ".gi-loading"
    ]

    end = time.time() + timeout

    while time.time() < end:
        still_visible = False
        for css in possible_loaders:
            try:
                el = driver.find_element(By.CSS_SELECTOR, css)
                if el.is_displayed():
                    still_visible = True
                    break
            except:
                continue
        if not still_visible:
            print("✅ 页面无 loading 遮挡")
            return
        time.sleep(0.5)

    print("⚠️ 超时仍检测到 loading，继续执行（可能已隐藏）")


In [ ]:
def wait_for_mask_to_disappear(timeout=30):
    print("⏳ 等待遮罩/弹层消失...")

    # 1) 等页面 ready
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )

    # 2) 通用遮罩 / loading / 弹层 选择器（可按需增减）
    mask_xpath = """
    //div[
        contains(@class,'loading')
        or contains(@class,'spinner')
        or contains(@class,'mask')
        or contains(@class,'overlay')
        or contains(@class,'modal')
        or contains(@class,'dialog')
        or contains(@class,'backdrop')
        or contains(@style,'z-index')
    ]
    """

    try:
        # 等所有可能的遮罩元素“不可见”
        WebDriverWait(driver, timeout).until(
            lambda d: all(
                not el.is_displayed()
                for el in d.find_elements(By.XPATH, mask_xpath)
            )
        )
        print("✅ 遮罩已消失")
    except:
        print("⚠️ 未检测到遮罩或超时（可能没有遮罩/类名变化），继续执行")


In [ ]:
def wait_for_loading_to_disappear(timeout=60):
    print("⏳ 等待页面 loading 消失...")

    WebDriverWait(driver, timeout).until(
        EC.invisibility_of_element_located((
            By.XPATH,
            "//*[contains(@class,'loading') or contains(@class,'spinner') or contains(@class,'mask') or contains(@class,'overlay')]"
        ))
    )

    print("✅ loading 已消失")

def click_with_retry(xpath, name, retries=3, wait_sec=30):
    """
    稳定点击封装：等待 + 重试 + 超时控制
    """
    for i in range(retries):
        try:
            btn = WebDriverWait(driver, wait_sec).until(
                EC.element_to_be_clickable((By.XPATH, xpath))
            )
            wait_for_loading_to_disappear()
            btn.click()
            print(f"✅ 点击成功：{name}")
            return
        except Exception as e:
            print(f"⏳ {name} 第 {i+1} 次点击失败，重试中...", repr(e))
            time.sleep(3)
    raise TimeoutException(f"{name} 多次点击失败")


In [ ]:
def click_left_block():
    print("🖱️ 点击真正的内容模块（生日文案）...")

    wait_for_loading_to_disappear()

    # 🔥 精确锁定包含文案的段落
    p = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((
            By.XPATH,
            "//p[contains(., '亲爱的月捐伙伴') and contains(., '生日')]"
        ))
    )

    # 👇 往上找最近可点击的大容器（真正的块）
    block = p.find_element(
        By.XPATH,
        "ancestor::div[contains(@class,'content') or contains(@class,'block') or contains(@class,'wrapper') or contains(@class,'tmp') or contains(@class,'ui-draggable')][1]"
    )

    # 滚到可见
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", block)
    time.sleep(0.5)

    # 👇 强制 JS 点击
    driver.execute_script("arguments[0].click();", block)

    print("✅ 模块已点击，应已进入右侧编辑态")


def switch_to_editor_iframe():
    iframe = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, "//iframe"))
    )
    driver.switch_to.frame(iframe)
    


In [ ]:
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def edit_right_editor(name, month, day):
    print("✏️ 在右侧编辑器中写入真实内容...")

    date_md = f"{month}月{day:02d}日"
    year = datetime.now().year
    date_ymd = f"{year}年{month}月{day:02d}日"

    # 1️⃣ 称呼段落
    p_name = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.XPATH, "//p[contains(., '亲爱的月捐伙伴')]")
        )
    )

    # 2️⃣ “生日快乐”段落
    p_birthday = driver.find_element(
        By.XPATH,
        "//p[contains(., '生日') and contains(., 'PEER祝你生日快乐')]"
    )

    # 3️⃣ 署名下面的日期（最后一个带“年/月”的 <p>）
    p_dates = driver.find_elements(
        By.XPATH,
        "//p[contains(., '年') and contains(., '月')]"
    )
    if not p_dates:
        raise RuntimeError("找不到包含日期（年/月）的段落")
    p_date_tail = p_dates[-1]

    # 小工具：同时改内容 + 颜色 + 加粗
    def set_p(el, text):
        driver.execute_script("""
            arguments[0].innerHTML = arguments[1];
            arguments[0].style.color = 'rgb(48,151,102)';   // #309766
            arguments[0].style.fontWeight = 'bold';
        """, el, text)

    # === 写回内容并上色 ===
    set_p(p_name, f"亲爱的月捐伙伴 {name}，")
    set_p(p_birthday, f"{date_md}是你的生日，PEER祝你生日快乐！")
    set_p(p_date_tail, date_ymd)

    print("✅ 右侧文本已写入并加粗上色")
    driver.switch_to.default_content()


In [ ]:


def edit_template_for_user(name, month, day):

    click_left_block()              # ✅ 先点左侧模块
    switch_to_editor_iframe()       # ✅ 进入右侧 iframe
    edit_right_editor(name, month, day)  # ✅ 改真实内容

    # ✅ 保存
    save_btn = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.ID, "manualSave"))
)

# ✅ 关键：等遮罩真正消失
    wait_for_mask_to_disappear()

    # ✅ 再确认按钮可点
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "manualSave")))

    # ✅ 再点击
    save_btn.click()
    print("💾 已点击保存")

    time.sleep(2)

    # ✅ 进入发送页
    click_with_retry("//a[@id='immediate-sent']", "发送按钮", retries=4, wait_sec=60)


In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time


def fill_email_and_schedule(email, send_datetime):
    print("📧 设置发送对象：", email)

    INPUT_XPATH = "//input[contains(@class,'lx-cp-search-bar__input')]"
    RESULT_XPATH = f"//span[contains(@class,'lx-cp-check-box__text') and normalize-space(text())='{email}']"
    SCHEDULE_XPATH = "//span[contains(text(),'定时发送')]/ancestor::div[contains(@class,'lx-checkbox')]"
    TIME_XPATH = "//input[contains(@class,'flatpickr-input')]"
    CONFIRM_SEND_XPATH = "//div[contains(@class,'btn-primary') and normalize-space()='确定']"

    SEND_XPATH = "//a[contains(@class,'btn-lg') and text()='发送']"

    # ============================
    def wait_for_mask(timeout=5):
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )
        try:
            WebDriverWait(driver, timeout).until(lambda d: all(
                not e.is_displayed() for e in
                d.find_elements(By.XPATH,
                    "//*[contains(@class,'loading') or contains(@class,'spinner') "
                    "or contains(@class,'mask') or contains(@class,'overlay') or contains(@class,'modal')]"
                )
            ))
        except:
            pass


    def switch_to_frame(xpath, timeout=15):
        driver.switch_to.default_content()
        try:
            WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH, xpath)))
            return
        except:
            pass
        for f in driver.find_elements(By.TAG_NAME, "iframe"):
            driver.switch_to.default_content()
            driver.switch_to.frame(f)
            try:
                WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH, xpath)))
                return
            except:
                continue
        raise TimeoutException("未找到控件（主页面 + iframe）")


    def js_type(el, value):
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
        driver.execute_script("arguments[0].focus();", el)
        driver.execute_script("arguments[0].value = '';", el)
        driver.execute_script("arguments[0].value = arguments[1];", el, value)
        driver.execute_script("""
            arguments[0].dispatchEvent(new Event('input', {bubbles:true}));
            arguments[0].dispatchEvent(new Event('change', {bubbles:true}));
        """, el)


    # ============================
    print("🛬 等待发送页加载...")
    wait_for_mask()

    WebDriverWait(driver, 20).until(
        EC.any_of(
            EC.presence_of_element_located((By.XPATH, INPUT_XPATH)),
            EC.presence_of_element_located((By.XPATH, "//iframe"))
        )
    )

    print("🧭 定位输入框...")
    switch_to_frame(INPUT_XPATH)


    # ===== 1. 输入邮箱 =====
    search = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, INPUT_XPATH)))
    try:
        search.clear()
        search.send_keys(email)
        search.send_keys(Keys.ENTER)
    except:
        js_type(search, email)
    time.sleep(2)


    # ===== 2. 选联系人 =====
    print("🔍 选择联系人...")
    try:
        contact = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, RESULT_XPATH))
        )
    except:
        prefix = email.split("@")[0]
        contact = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, f"//span[contains(text(),'{prefix}')]"))
        )
    ActionChains(driver).move_to_element(contact).pause(0.2).click().perform()


    # ===== ✅ 3. 勾选「定时发送」=====
    print("⏰ 勾选定时发送...")

    SCHEDULE_TEXT_XPATH = "//span[normalize-space()='定时发送']"

    schedule_label = WebDriverWait(driver, 40).until(
        EC.presence_of_element_located((By.XPATH, SCHEDULE_TEXT_XPATH))
    )

    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", schedule_label)
    time.sleep(0.3)

    clicked = False

    # ✅ 方式1：原生点击
    try:
        schedule_label.click()
        print("✅ label.click() 成功")
        clicked = True
    except Exception as e:
        print("⚠ label.click 失败：", e)

    # ✅ 方式2：JS click()
    if not clicked:
        try:
            driver.execute_script("arguments[0].click();", schedule_label)
            print("✅ JS click() label 成功")
            clicked = True
        except Exception as e:
            print("⚠ JS click 失败：", e)

    # ✅ 方式3：MouseEvent 强制触发
    if not clicked:
        print("⚠ 使用 MouseEvent 强制点击")
        driver.execute_script("""
            var el = arguments[0];
            function fire(type) {
                var evt = new MouseEvent(type, {bubbles:true, cancelable:true, view:window});
                el.dispatchEvent(evt);
            }
            fire('mouseover'); fire('mousedown'); fire('mouseup'); fire('click');
        """, schedule_label)

    print("✅ 定时发送点击完成")



    # ===== 4. 写时间（不确认） =====
    print("🕒 填写定时时间...")
    time_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, TIME_XPATH))
    )
    time_str = send_datetime.strftime("%Y-%m-%d %H:%M")
    js_type(time_input, time_str)
    time.sleep(1)

    # 若弹窗挡住“发送”，点击空白处关闭
    driver.execute_script("document.body.click();")
    time.sleep(0.5)

    # ===== ✅ 5. 点击发送按钮 =====
    print("🚀 点击发送按钮...")

    send_btn = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, SEND_XPATH))
    )

    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", send_btn)
    time.sleep(0.3)

    clicked = False

    # ✅ 方式1：原生点击
    try:
        send_btn.click()
        print("✅ send_btn.click() 成功")
        clicked = True
    except Exception as e:
        print("⚠ send_btn.click 失败：", e)

    # ✅ 方式2：JS click
    if not clicked:
        try:
            driver.execute_script("arguments[0].click();", send_btn)
            print("✅ JS click() 发送成功")
            clicked = True
        except Exception as e:
            print("⚠ JS click 失败：", e)

    # ✅ 方式3：MouseEvent
    if not clicked:
        print("⚠ 启用 MouseEvent 强制点击发送")
        driver.execute_script("""
            var el = arguments[0];
            function fire(type){
                var e = new MouseEvent(type,{bubbles:true,cancelable:true,view:window});
                el.dispatchEvent(e);
            }
            fire('mouseover');fire('mousedown');fire('mouseup');fire('click');
        """, send_btn)
        print("✅ MouseEvent 已触发发送")
        
    # ===== ✅ 6. 等待并点击确认 =====
    print("✅ 等待『发送确认』弹窗出现...")

    confirm = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.XPATH, CONFIRM_SEND_XPATH))
    )

    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", confirm)
    time.sleep(0.3)

    clicked = False

    try:
        driver.execute_script("arguments[0].click();", confirm)
        print("✅ 已确认发送")
        clicked = True
    except Exception as e:
        print("⚠ JS 点击失败：", e)

    if not clicked:
        confirm.click()
        print("✅ 原生 confirm.click 成功")

    # ===== ✅ 7. 第二次『确认操作』弹窗处理 =====
    print("✅ 等待第二个确认弹窗出现...")

    SECOND_CONFIRM_XPATH = "//div[contains(@class,'btn-primary') and normalize-space()='确认']"

    second_confirm = WebDriverWait(driver, 60).until(
        EC.element_to_be_clickable((By.XPATH, SECOND_CONFIRM_XPATH))
    )

    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", second_confirm)
    time.sleep(0.3)

    clicked = False

    # ✅ 首选：JS click（最稳）
    try:
        driver.execute_script("arguments[0].click();", second_confirm)
        print("✅ 第二个确认框：JS 点击成功")
        clicked = True
    except Exception as e:
        print("⚠ 第二次 JS 点击失败：", e)


    # ✅ 备用：原生 click
    if not clicked:
        try:
            second_confirm.click()
            print("✅ 第二个确认框：原生点击成功")
            clicked = True
        except Exception as e:
            print("⚠ 原生点击失败：", e)


    # ✅ 底层兜底：MouseEvent
    if not clicked:
        print("⚠ 使用 MouseEvent 强制确认第二个弹窗")
        driver.execute_script("""
            var el = arguments[0];
            function fire(type){
                var e = new MouseEvent(type,{bubbles:true,cancelable:true,view:window});
                el.dispatchEvent(e);
            }
            fire('mouseover');fire('mousedown');fire('mouseup');fire('click');
        """, second_confirm)
        print("✅ 第二确认弹窗 MouseEvent 已触发")

    print("🎉✅ 所有确认已完成，定时发送成功")


    print("🎉 发送流程完成")


In [ ]:
def switch_to_latest_window(timeout=10):
    import time
    time.sleep(1)  # 给浏览器1秒创建新窗口

    all_windows = driver.window_handles
    print("🪟 当前已打开窗口数：", len(all_windows))

    # 默认切到最后一个新开窗口
    driver.switch_to.window(all_windows[-1])
    print("✅ 已切换至最新窗口：", driver.current_url)


In [ ]:
from datetime import datetime
import time

SUCCESS = 0
FAILED = 0
failed_list = []

print("🚀 开始批量处理用户...")

for idx, row in df_dec.iterrows():
    print("\n" + "="*12 + f" 开始处理第 {idx} 条 " + "="*12)

    try:
        # 1. 读取基础信息
        name = row[name_col]
        email = row[email_col]
        month = int(row[month_col])
        day = int(row[day_col])

        print(f"✅ 姓名：{name}")
        print(f"✅ 邮箱：{email}")
        print(f"✅ 生日：{month}-{day:02d}")

        # 2. 计算发送时间（今年生日 00:00）
        send_year = datetime.now().year
        send_dt = datetime(send_year, month, day, 0, 0)
        print(f"⏰ 发送时间：{send_dt}")

        # 3. 打开模板页
        print("🌐 打开模板页面：", TEMPLATE_URL)
        driver.get(TEMPLATE_URL)
        time.sleep(2)

        # 4. 编辑模板
        print("📝 编辑模板中...")
        edit_template_for_user(name, month, day)
        print("✅ 模板编辑完成")

        # 5. 切到发送页（新标签）
        print("🪟 切到发送页...")
        switch_to_latest_window()
        time.sleep(2)
        print("当前发送页 URL：", driver.current_url)

        # 6. 填邮箱 + 定时发送
        print("📨 设置发送中...")
        fill_email_and_schedule(email, send_dt)
        print("✅ 定时发送已设置")

        SUCCESS += 1
        print("🎉 本条完成")

        # 小暂停，防止请求过快
        time.sleep(3)
        
    except Exception as e:
        FAILED += 1
        failed_list.append((idx, name, email, str(e)))
        print("❌ 处理失败：", repr(e))
        print("➡️ 跳过该用户，继续下一个...")
        time.sleep(3)
        continue

print("\n📊 批量完成统计")
print(f"✅ 成功：{SUCCESS}")
print(f"❌ 失败：{FAILED}")
print("🏁 全部处理结束")


In [ ]:
print("\n❌ 失败列表：")
for item in failed_list:
    print(item)
